In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from model_factories import get_golem_model, get_resnet18_model, get_resnet34_model, get_vitbase_model
from dataloaders import get_dataloaders
from model_utils import train_epoch, eval
from plotter import plot_images

In [2]:
device = torch.device(
        "mps"
        if torch.backends.mps.is_available()
        else "cuda" if torch.cuda.is_available() else "cpu"
    )
train_loader, test_loader, classes = get_dataloaders(test_run=True)
CLASSES_NUM = len(classes)

# Scenario 1

In [3]:
# Models
# gc = get_resnet18_model(CLASSES_NUM)
# gc = get_resnet34_model(CLASSES_NUM)
# gc = get_vitbase_model(CLASSES_NUM)
gc = get_golem_model(CLASSES_NUM)
gc.to(device)

# Loss functions
criterionCEL = nn.CrossEntropyLoss()
# criterionMSE = nn.MSELoss()

# Optimizers
optimizerAdam = optim.Adam(gc.parameters(), lr=1e-3)
# optimizerSGD = optim.SGD(gc.parameters(), lr=1e-3, momentum=0.9)
# optimizerRMS = optim.RMSprop(gc.parameters(), lr=1e-3, momentum=0.9)
# optimizerAdadelta = optim.Adadelta(gc.parameters(), lr=1e-3)
# optimizerAdamax = optim.Adamax(gc.parameters(), lr=1e-3)

In [4]:
for i in range(2):
    loss = train_epoch(gc, criterionCEL, optimizerAdam, train_loader, device)
    print(f"{i+1}/2: loss={loss}")

1/2: loss=3062.4492901563644
2/2: loss=2255.652959406376


In [5]:
metrics, preds = eval(gc, criterionCEL, test_loader, device)
loss, accuracy, precision, recall, f1 = metrics
print("Custom backbone + CELoss + AdamOptim results: ")
print(f"Avg loss {loss}\nAccu {accuracy}\nPrecision {precision}\nRecall {recall}\nF-score {f1}\n")

# plot_images(test_loader, preds, classes)

Custom backbone + CELoss + AdamOptim results: 
Avg loss 1.2750584326993923
Accu 0.5529999732971191
Precision 0.35
Recall 0.6833333333333333
F-score 0.3538095238095238



# Scenario 2

In [6]:
# Models
# gc = get_resnet18_model(CLASSES_NUM)
# gc = get_resnet34_model(CLASSES_NUM)
# gc = get_vitbase_model(CLASSES_NUM)
gc = get_golem_model(CLASSES_NUM)
gc.to(device)

# Loss functions
criterionCEL = nn.CrossEntropyLoss()
# criterionMSE = nn.MSELoss()

# Optimizers
# optimizerAdam = optim.Adam(gc.parameters(), lr=1e-3)
optimizerSGD = optim.SGD(gc.parameters(), lr=1e-3, momentum=0.9)
# optimizerRMS = optim.RMSprop(gc.parameters(), lr=1e-3, momentum=0.9)
# optimizerAdadelta = optim.Adadelta(gc.parameters(), lr=1e-3)
# optimizerAdamax = optim.Adamax(gc.parameters(), lr=1e-3)

In [7]:
for i in range(2):
    loss = train_epoch(gc, criterionCEL, optimizerSGD, train_loader, device)
    print(f"{i+1}/2: loss={loss}")

1/2: loss=2490.450904428959
2/2: loss=2129.4462230205536


In [8]:
metrics, preds = eval(gc, criterionCEL, test_loader, device)
loss, accuracy, precision, recall, f1 = metrics
print("Custom backbone + CELoss + SGDOptim results: ")
print(f"Avg loss {loss}\nAccu {accuracy}\nPrecision {precision}\nRecall {recall}\nF-score {f1}\n")

# plot_images(images, preds, classes)

Custom backbone + CELoss + SGDOptim results: 
Avg loss 1.2209272992115814
Accu 0.5794000029563904
Precision 0.5166666666666666
Recall 0.6833333333333333
F-score 0.3966666666666666



# Scenario 3

In [9]:
# Models
gc = get_resnet18_model(CLASSES_NUM)
# gc = get_resnet34_model(CLASSES_NUM)
# gc = get_vitbase_model(CLASSES_NUM)
# gc = get_golem_model(CLASSES_NUM)
gc.to(device)

# Loss functions
criterionCEL = nn.CrossEntropyLoss()
# criterionMSE = nn.MSELoss()

# Optimizers
optimizerAdam = optim.Adam(gc.parameters(), lr=1e-3)
# optimizerSGD = optim.SGD(gc.parameters(), lr=1e-3, momentum=0.9)
# optimizerRMS = optim.RMSprop(gc.parameters(), lr=1e-3, momentum=0.9)
# optimizerAdadelta = optim.Adadelta(gc.parameters(), lr=1e-3)
# optimizerAdamax = optim.Adamax(gc.parameters(), lr=1e-3)

In [10]:
for i in range(2):
    loss = train_epoch(gc, criterionCEL, optimizerAdam, train_loader, device)
    print(f"{i+1}/2: loss={loss}")

1/2: loss=2319.6230943202972
2/2: loss=1663.1560254096985


In [11]:
metrics, preds = eval(gc, criterionCEL, test_loader, device)
loss, accuracy, precision, recall, f1 = metrics
print("Resnet18 backbone + CELoss + AdamOptim results: ")
print(f"Avg loss {loss}\nAccu {accuracy}\nPrecision {precision}\nRecall {recall}\nF-score {f1}\n")

# plot_images(images, preds, classes)

Resnet18 backbone + CELoss + AdamOptim results: 
Avg loss 0.9501626108781979
Accu 0.6740999817848206
Precision 0.7111111111111111
Recall 0.6296296296296297
F-score 0.4222222222222222



Scenario 4

In [12]:
# Models
# gc = get_resnet18_model(CLASSES_NUM)
gc = get_resnet34_model(CLASSES_NUM)
# gc = get_vitbase_model(CLASSES_NUM)
# gc = get_golem_model(CLASSES_NUM)
gc.to(device)

# Loss functions
criterionCEL = nn.CrossEntropyLoss()
# criterionMSE = nn.MSELoss()

# Optimizers
optimizerAdam = optim.Adam(gc.parameters(), lr=1e-3)
# optimizerSGD = optim.SGD(gc.parameters(), lr=1e-3, momentum=0.9)
# optimizerRMS = optim.RMSprop(gc.parameters(), lr=1e-3, momentum=0.9)
# optimizerAdadelta = optim.Adadelta(gc.parameters(), lr=1e-3)
# optimizerAdamax = optim.Adamax(gc.parameters(), lr=1e-3)

In [13]:
for i in range(2):
    loss = train_epoch(gc, criterionCEL, optimizerAdam, train_loader, device)
    print(f"{i+1}/2: loss={loss}")

1/2: loss=2539.5317855477333
2/2: loss=1935.6963964700699


In [14]:
metrics, preds = eval(gc, criterionCEL, test_loader, device)
loss, accuracy, precision, recall, f1 = metrics
print("Resnet34 backbone + CELoss + AdamOptim results: ")
print(f"Avg loss {loss}\nAccu {accuracy}\nPrecision {precision}\nRecall {recall}\nF-score {f1}\n")

# plot_images(images, preds, classes)

Resnet34 backbone + CELoss + AdamOptim results: 
Avg loss 1.1395214361885486
Accu 0.5978000164031982
Precision 0.7416666666666667
Recall 0.625
F-score 0.4770833333333333



Scenario 5

In [15]:
# Models
# gc = get_resnet18_model(CLASSES_NUM)
# gc = get_resnet34_model(CLASSES_NUM)
gc = get_vitbase_model(CLASSES_NUM)
# gc = get_golem_model(CLASSES_NUM)
gc.to(device)

# Loss functions
criterionCEL = nn.CrossEntropyLoss()
# criterionMSE = nn.MSELoss()

# Optimizers
optimizerAdam = optim.Adam(gc.parameters(), lr=1e-3)
# optimizerSGD = optim.SGD(gc.parameters(), lr=1e-3, momentum=0.9)
# optimizerRMS = optim.RMSprop(gc.parameters(), lr=1e-3, momentum=0.9)
# optimizerAdadelta = optim.Adadelta(gc.parameters(), lr=1e-3)
# optimizerAdamax = optim.Adamax(gc.parameters(), lr=1e-3)

In [17]:
for i in range(2):
    loss = train_epoch(gc, criterionCEL, optimizerAdam, train_loader, device)
    print(f"{i+1}/2: loss={loss}")

AssertionError: Wrong image height! Expected 224 but got 32!

In [ ]:
metrics, preds = eval(gc, criterionCEL, test_loader, device)
loss, accuracy, precision, recall, f1 = metrics
print("Vitbase backbone + CELoss + AdamOptim results: ")
print(f"Avg loss {loss}\nAccu {accuracy}\nPrecision {precision}\nRecall {recall}\nF-score {f1}\n")

# plot_images(images, preds, classes)